For this example, following are the **assumptions**.


1.   **Time Series** data: we have two time series data points
- a. Inpatient data i.e number of patients requiring hospitalization on daily basis.
- b. Outpatient data i.e number of patients who just need to consult doctor (no hospitalization) on daily basis.

2.   **Non-Time Series** data: We also have two type of non time-series data as well.
- Day of the week (value can be 0 to 6)
- Holiday (value can be 0 or 1)

In the code below, 

1. we will use **two LSTMs** (one each for Inpatient and outpatient data) to understand two time series data features. To each LSTM, we will feed a sequence of information (last few days of inpatient/outpatient data).

2. The output of LSTMs will be **concatenated** with non-time series data features and then passed through other hidden layer(s) and output layer.


As we are working with multiple inputs, this will be a non-sequential model. In non-sequential model, we build model layer by layer starting with 'InputLayer' for each input.


In [1]:
import tensorflow as tf

Working with inpatient data:

1. Create input layer to feed inpatient data. Assuming that last 5 days data is being used to prepare a sequence i.e **window size is 5**. 
2. Input layer data is first reshaped to **3D** (for LSTM) and then fed to LSTM. 
3. LSTM ouput will be **hidden state** at last time step (after working with all 5 data elements in the sequence).

In [2]:
#Inpatient
input_1 = tf.keras.layers.Input(shape=(5,))

In [3]:
#lstm for inpatient
x1 = tf.keras.layers.Reshape((5,1,))(input_1)
lstm1 = tf.keras.layers.LSTM(60)(x1)

In [4]:
lstm1

<KerasTensor: shape=(None, 60) dtype=float32 (created by layer 'lstm')>

In [5]:
x1

<KerasTensor: shape=(None, 5, 1) dtype=float32 (created by layer 'reshape')>

Here we will process Outpatient time series data
1. Input layer where a sequence of 3 data elements (**window size = 3**) is fed.
2. Reshape layer to make it 3D for LSTM
3. LSTM layer which will provide last hidden state as output

In [6]:
#outatient
input_2 = tf.keras.layers.Input(shape=(3,))

In [7]:
#lstm for outpatient
x2 = tf.keras.layers.Reshape((3,1,))(input_2)
lstm2 = tf.keras.layers.LSTM(30)(x2)

In [8]:
lstm2

<KerasTensor: shape=(None, 30) dtype=float32 (created by layer 'lstm_1')>

Input layer for
1. **Weekday** data: for each example, we will feed 7 numbers (**One hot encoding**)
2. **Holiday**: Feed one number per example (either 0 or 1)

In [9]:
#Weekday
input_3 = tf.keras.layers.Input(shape=(7,))

In [10]:
#Holiday
input_4 = tf.keras.layers.Input(shape=(1,))

**Concatenate** output of two LSTMs with two non time-series features

In [11]:
concat = tf.keras.layers.concatenate([lstm1, lstm2, input_3, input_4])

In [12]:
concat

<KerasTensor: shape=(None, 98) dtype=float32 (created by layer 'concatenate')>

**Output layer** (you can have other hidden layers before output layer)

In [13]:
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(concat)

In [14]:
output_layer

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense')>

Build **non-sequential model** by indicating all inputs and output layers.

In [15]:
#4 input layers and 1 output layer
model = tf.keras.Model([input_1, input_2, input_3, input_4], output_layer)

In [16]:
#Compile the model
model.compile(optimizer='adam', loss='mse')

In [17]:
#Model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 5, 1)         0           input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 3, 1)         0           input_2[0][0]                    
______________________________________________________________________________________________